This is code that we use in Smart Warehouse Hackathon which contain:
- Create Train datasets
- Augmentation
- Train
- Predict
    - pose precess
    - combind and create submitsion file
- Ensemble

Note: We do those task separately but we try to write everything in one notebook. Consequently, it might has some error in directory because we didnot do everything by coding.

Example videos:

Short: https://drive.google.com/file/d/1urocaVfrtTtZGCTDlzOk8ybIs1Q2SR5H/view?usp=sharing

Long: https://drive.google.com/file/d/15dy1OjI30Kyn5IF8_O5kM0G1FXB7GvCp/view?usp=sharing

Note: Short video will compare normal detection and detection with motion detection.

authors:
- Patchara Opaspilai
- Pitikorn Khlaisamniang

# [1] Create Train Dataset Format for YOLO

In [ ]:
# We assume that you are already download and unzip train and test files

import os
import pandas as pd
import cv2

path_annotations_front_left = "train/train/annotations/bangphli_front_left/"
path_annotations_front_center = "train/train/annotations/bangphli_front_center/"
path_video_front_left = "train/train/videos/bangphli_front_left/"
path_video_front_center = "train/train/videos/bangphli_front_center/"

Path_annotations = [path_annotations_front_left, path_annotations_front_center]
Path_videos = [path_video_front_left, path_video_front_center]

for p in range(len(Path_annotations)):
    path_annotations = Path_annotations[p]
    path_videos = Path_videos[p]
    for filename in os.listdir(path_annotations):
        df = pd.read_csv(path_annotations + filename + "/annotations.csv")

        # Save images
        frame_list = []
        for frame in df["frame"]:
            frame_list.append(frame)
        video_path = path_videos + filename + "/" + filename + ".mp4"
        vidcap = cv2.VideoCapture(video_path)
        success = True
        width = vidcap.get(3)
        height = vidcap.get(4)
        count = 0
        while success:
            success, image = vidcap.read()
            if count in frame_list:
                try:
                    cv2.imwrite(
                        "Train/images/" + filename + "_" + str(count) + ".jpg", image
                    )  # save frame as JPEG file
                except:
                    print(filename + "_" + str(count))
                    print(success)
            count += 1

        # Save labels
        for i in range(len(df)):
            txt = (
                "0 "
                + str(df["centroid_x"][i] / width)
                + " "
                + str(df["centroid_y"][i] / height)
                + " "
                + str(df["width"][i] / width)
                + " "
                + str(df["height"][i] / height)
            )

            with open("Train/labels/" + filename + "_" + str(df["frame"][i]) + ".txt", "w") as f:
                f.write(txt)

# [2] Augmentation

In [ ]:
import albumentations as A
import os
import cv2
import shutil

path = "Train/"

image_list = os.listdir(path + "images/")

os.mkdir("Results")

source_dir = path
destination_dir = "Results"
shutil.copytree(source_dir, destination_dir)
C = 0
for name in image_list:
    if C % 2 == 0:
        image = cv2.imread(path + "images/" + name)
        w = image.shape[1]
        h = image.shape[0]
        txt = name[:-4] + ".txt"
        f = open(path + "labels/" + txt, "r")
        bboxes = []
        for x in f:
            Class = str(x.split(" ")[0])
            X_c = float(x.split(" ")[1])
            Y_c = float(x.split(" ")[2])
            W = float(x.split(" ")[3])
            H = float(x.split(" ")[4])
            bboxes.append([X_c, Y_c, W, H, Class])

        transform = A.Compose(
            [
                # A.RandomCrop(width=450, height=450),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.7),
            ],
            bbox_params=A.BboxParams(format="yolo"),
        )

        transformed = transform(image=image, bboxes=bboxes)
        transformed_image = transformed["image"]
        transformed_bboxes = transformed["bboxes"]

        cv2.imwrite(destination_dir + "/images/" + name[:-4] + "_2" + ".jpg", transformed_image)
        labels = (
            str(transformed_bboxes[0][4])
            + " "
            + str(transformed_bboxes[0][0])
            + " "
            + str(transformed_bboxes[0][1])
            + " "
            + str(transformed_bboxes[0][2])
            + " "
            + str(transformed_bboxes[0][3])
        )

        with open(destination_dir + "/labels/" + name[:-4] + "_2" + ".txt", "w") as f:
            f.write(labels)
    C = C + 1

# [3] Train

Create data.yaml

In [ ]:
# We split data into 5 splits for train in 5 kfolds but we train only one split because there is no time

# Create data.yaml

import yaml

data = ""

with open("data.yaml", "w") as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

# Copy and paste this in yaml
"""
names:
  0: 0
path: Result
train: train
val: val
"""

Split data

In [ ]:
import ultralytics

ultralytics.checks()
import datetime
import shutil
from pathlib import Path
from collections import Counter

import yaml
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.model_selection import KFold

In [ ]:
dataset_path = Path("Result")  # replace with 'path/to/dataset' for your custom data
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'

In [ ]:
yaml_file = "data.yaml"  # your data YAML with data directories and names dictionary
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes.keys())

In [ ]:
indx = [l.stem for l in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [ ]:
for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for l in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(l.split(" ")[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

In [ ]:
ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [ ]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{idx}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{idx}"].loc[labels_df.iloc[val].index] = "val"

In [ ]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

In [ ]:
supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump({"path": split_dir.as_posix(), "train": "train", "val": "val", "names": classes}, ds_y)

In [ ]:
for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

In [ ]:
folds_df.to_csv(save_path / "kfold_datasplit.csv")
fold_lbl_distrb.to_csv(save_path / "kfold_label_distribution.csv")

Train 3 models

In [ ]:
# yolov8m

from ultralytics import YOLO

# Load a model
model = YOLO("yolov8m.pt")

# Train the model
results = model.train(
    data="Result/2023-11-08_5-Fold_Cross-val/split_1/split_1_dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=-1,
    patience=20,
    name="yolov8",
)

In [ ]:
# yolov5m (This model is for 1280 image size)

# Load a model
model = YOLO("yolov5m6u.pt")

# Train the model
results = model.train(
    data="Result/2023-11-08_5-Fold_Cross-val/split_1/split_1_dataset.yaml",
    epochs=100,
    imgsz=1280,
    batch=-1,
    patience=20,
    name="yolov5",
)

In [ ]:
# yolov6m

# Load a model
model = YOLO("yolov6m.yaml")

# Train the model
results = model.train(
    data="Result/2023-11-08_5-Fold_Cross-val/split_1/split_1_dataset.yaml",
    epochs=100,
    imgsz=640,
    batch=-1,
    patience=20,
    name="yolov6",
)

# [4] Predict

- We predict and create CSV for each videos and do pose process then combine all together.

- In order to track the objects we use track form ultralytics. There will be a lot of box that get detect and track but not our targets. (Our targets is boxes that lifted by forklift) Therefore, we use motion detection to filter boxes that does not move by correct only id that pass the filter.
- After predicr, we do pose process for clear all data that we dont want. For instance, id that we dont pass our filter. Moreover, rearange number of id.
- Then we combine all csv together and use sequence_to_track file to create predicted_output by matching frame, video_name in combine_csv and sequence_to_track.

Create csv

In [ ]:
import cv2, time
from ultralytics import YOLO
from shapely.geometry import Polygon
import pandas as pd
import os

In [ ]:
def calculate_iou(box_1, box_2):
    poly_1 = Polygon(box_1)
    poly_2 = Polygon(box_2)
    iou = poly_1.intersection(poly_2).area / poly_1.union(poly_2).area
    return iou


def box2box(boxes):
    xmin = float(boxes[0])
    ymin = float(boxes[1])
    xmax = float(boxes[0]) + float(boxes[2])
    ymax = float(boxes[1]) + float(boxes[3])

    return [[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]]


def box2box_yolo(boxes):
    xmin = float(boxes[0]) - float(boxes[2]) / 2
    ymin = float(boxes[1]) - float(boxes[3]) / 2
    xmax = float(boxes[0]) + float(boxes[2]) / 2
    ymax = float(boxes[1]) + float(boxes[3]) / 2

    return [[xmin, ymin], [xmax, ymin], [xmax, ymax], [xmin, ymax]]

In [ ]:
# Predict ones per model

model_name = "yolov8m"
path = "test/test/"
os.mkdir("csv")
os.mkdir("csv/" + model_name)
test_path = os.listdir(path)
model = YOLO("runs/detect/" + model_name + "/weights/best.pt")
font = cv2.FONT_HERSHEY_SIMPLEX
Video_count = 0

Done_list = []


for test_floder in test_path:
    type_path = os.listdir(path + test_floder + "/test/videos/")
    for type in type_path:
        video_path = path + test_floder + "/test/videos/" + type
        videos = os.listdir(video_path)
        for video in videos:
            if Video_count not in Done_list:
                # print(Video_count)
                # print(video_path+'/'+video)
                # print(type,video)
                first_frame = None
                check = True
                Match = False
                Old_COUNT = []
                COUNT = []
                COUNT_ID = []
                Frame_count = 0
                df = pd.DataFrame()
                TYPE = []
                VIDEO_NAME = []
                ID = []
                FRAME = []
                XMIN = []
                YMIN = []
                XMAX = []
                YMAX = []

                Video = cv2.VideoCapture(video_path + "/" + video)

                while check:
                    check, frame = Video.read()
                    if check == False:
                        break
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    gray = cv2.GaussianBlur(gray, (21, 21), 0)
                    if first_frame is None:
                        first_frame = gray
                        continue
                    delta_frame = cv2.absdiff(first_frame, gray)
                    threshold_frame = cv2.threshold(delta_frame, 50, 255, cv2.THRESH_BINARY)[1]
                    threshold_frame = cv2.dilate(threshold_frame, None, iterations=2)

                    (cntr, _) = cv2.findContours(threshold_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    Moving_area = []

                    for contour in cntr:
                        if cv2.contourArea(contour) < 1000:
                            continue
                        (x, y, w, h) = cv2.boundingRect(contour)
                        Moving_area.append(box2box([x, y, w, h]))
                        # cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)

                    results = model.track(frame, persist=True, tracker="bytetrack.yaml", imgsz=640)

                    # Visualize the results on the frame
                    annotated_frame = results[0].plot()

                    boxes = results[0].boxes.xywh.cpu()
                    track_ids = results[0].boxes.id

                    for i in range(len(boxes)):
                        Match = False
                        M = 0
                        for j in range(len(Moving_area)):
                            if calculate_iou(box2box_yolo(boxes[i]), Moving_area[j]) > 0.0:
                                Match = True
                                M = M + calculate_iou(box2box_yolo(boxes[i]), Moving_area[j])

                        if Match == True and M > 0.1:
                            if track_ids == None:
                                ids = "-"
                            else:
                                ids = int(track_ids[i])

                                if len(COUNT) <= ids:
                                    for k in range(ids - len(COUNT) + 1):
                                        COUNT.append(0)
                                    for l in range(ids - len(COUNT_ID) + 1):
                                        COUNT_ID.append(0)
                                COUNT[ids] = COUNT[ids] + 1

                            xmin = float(boxes[i][0]) - float(boxes[i][2]) / 2
                            ymin = float(boxes[i][1]) - float(boxes[i][3]) / 2
                            xmax = float(boxes[i][0]) + float(boxes[i][2]) / 2
                            ymax = float(boxes[i][1]) + float(boxes[i][3]) / 2
                            # print(ids)
                            if ids == "-":
                                color = (0, 0, 255)
                            elif COUNT[ids] < 15:
                                color = (0, 0, 255)
                            else:
                                color = (0, 255, 0)
                                COUNT_ID[ids] = 1
                            cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 3)

                            frame = cv2.putText(
                                frame,
                                "ID" + str(ids) + "_" + str(M),
                                (int(xmin), int(ymin) + 5),
                                font,
                                2,
                                color,
                                5,
                                cv2.LINE_AA,
                            )
                            TYPE.append(type)
                            VIDEO_NAME.append(video)
                            ID.append(ids)
                            FRAME.append(Frame_count)
                            XMIN.append(int(xmin))
                            YMIN.append(int(ymin))
                            XMAX.append(int(xmax))
                            YMAX.append(int(ymax))

                    if Frame_count % 100 == 0:
                        for z in range(len(Old_COUNT)):
                            if Old_COUNT[z] == COUNT[z] and COUNT[z] < 15:
                                COUNT[z] = 0

                        Old_COUNT = COUNT

                    if Frame_count % 1000 == 0:
                        df = pd.DataFrame()
                        df["type"] = TYPE
                        df["name"] = VIDEO_NAME
                        df["id"] = ID
                        df["frame"] = FRAME
                        df["xmin"] = XMIN
                        df["ymin"] = YMIN
                        df["xmax"] = XMAX
                        df["ymax"] = YMAX
                        df.to_csv("watch.csv", index=False)
                    print(COUNT)
                    print(COUNT_ID)
                    Ans = []
                    for m in range(len(COUNT_ID)):
                        if COUNT_ID[m] == 1:
                            Ans.append(m)
                    print(Ans)
                    frame = cv2.putText(frame, str(Ans), (0, 100), font, 2, (255, 0, 0), 5, cv2.LINE_AA)
                    # cv2.imshow("YOLOv8 Tracking", frame)
                    Frame_count = Frame_count + 1
                    # cv2.imshow("frame",frame)
                    # key=cv2.waitKey(1)
                    # if key==ord('q'):
                    #    break

                Video.release()
                cv2.destroyAllWindows()
                df = pd.DataFrame()
                df["type"] = TYPE
                df["name"] = VIDEO_NAME
                df["id"] = ID
                df["frame"] = FRAME
                df["xmin"] = XMIN
                df["ymin"] = YMIN
                df["xmax"] = XMAX
                df["ymax"] = YMAX

                for y in range(len(COUNT)):
                    if COUNT_ID[y] == 0:
                        df = df.drop(df[df["id"] == y].index)

                for i in range(len(Ans)):
                    df.loc[df["id"] == Ans[i], "id"] = i

                df = df.drop(df[df["id"] == "-"].index)

                df.to_csv("csv/" + model_name + "/" + str(Video_count) + ".csv", index=False)

            Video_count = Video_count + 1

Pose precess

In [ ]:
# replace all csv by Pose precess csv

import os
import pandas as pd

model_name = "yolov8m"

path = "csv/" + model_name + "/"

csv_path = os.listdir(path)


for i in range(len(csv_path)):
    df_csv = pd.read_csv(path + str(i) + ".csv")
    Del = []
    Start = -1
    for j in range(len(df_csv)):
        if df_csv["id"][j] == Start or df_csv["id"][j] == Start + 1:
            Del.append(0)
            Start = df_csv["id"][j]
        else:
            Del.append(1)
    df_csv["Del"] = Del
    df_csv = df_csv.drop(df_csv[df_csv["Del"] == 1].index)
    df_csv = df_csv.drop(columns=["Del"])
    df_csv.reset_index(inplace=True)
    MAX = max(df_csv["id"])
    Dis = []
    for k in range(MAX + 1):
        First = True
        for l in range(len(df_csv)):
            if df_csv["id"][l] == k:
                if First == True:
                    first = [df_csv["xmax"][l] - df_csv["xmin"][l], df_csv["ymax"][l] - df_csv["ymin"][l]]
                    First = False

                last = [df_csv["xmax"][l] - df_csv["xmin"][l], df_csv["ymax"][l] - df_csv["ymin"][l]]
        distance = abs(last[0] - first[0]) ** 2 + abs(last[1] - first[1]) ** 2
        Dis.append(distance)

    Dis.reverse()
    for m in reversed(range(len(Dis))):
        distance = Dis[MAX - m]
        if distance <= 1000:
            df_csv = df_csv.drop(df_csv[df_csv["id"] == m].index)

            for n in range(MAX - m):
                # print((str(MAX-n)+' to ' + str(MAX-n-1)))
                df_csv.loc[df_csv["id"] == MAX - n, "id"] = MAX - n - 1

    df_csv.reset_index(inplace=True)

    df_csv = df_csv.drop(columns=["index", "level_0"])
    df_csv.to_csv("csv/" + model_name + "/" + str(i) + ".csv", index=False)

Combine csv

In [ ]:
import os
import pandas as pd

model_name = "yolov8m"

path = "csv/" + model_name + "/"

csv_path = os.listdir(path)

frames = []

for i in range(len(csv_path)):
    df_csv = pd.read_csv(path + str(i) + ".csv")
    frames.append(df_csv)

result = pd.concat(frames)

result.to_csv(model_name + ".csv", index=False)

fin boxes column

In [ ]:
import pandas as pd

model_name = "yolov8m"

df = pd.read_csv("sequence_to_track.csv")
df_ans = pd.read_csv(model_name + ".csv")

xmin = []
ymin = []
xmax = []
ymax = []

for i in range(len(df)):
    Find = False
    check = df["name"][i] + str(".mp4")
    df_check = df_ans.loc[(df_ans["name"] == check)]
    df_check = df_check.reset_index()
    for j in range(len(df_check)):
        if df["frame"][i] == df_check["frame"][j]:
            Find = True
            Xmin = df_check["xmin"][j]
            Ymin = df_check["ymin"][j]
            Xmax = df_check["xmax"][j] - df_check["xmin"][j]
            Ymax = df_check["ymax"][j] - df_check["ymin"][j]

    if Find == True:
        xmin.append(Xmin)
        ymin.append(Ymin)
        xmax.append(Xmax)
        ymax.append(Ymax)

    else:
        if df["xmin"][i] == -1:
            xmin.append(-1)
            ymin.append(-1)
            xmax.append(-1)
            ymax.append(-1)
        else:
            xmin.append(df["xmin"][i])
            ymin.append(df["ymin"][i])
            xmax.append(df["xmax"][i])
            ymax.append(df["ymax"][i])


df["xmin"] = xmin
df["ymin"] = ymin
df["xmax"] = xmax
df["ymax"] = ymax


df.to_csv(model_name + "_before.csv", index=False)

Another pose precess
- These are some frames that we cant detect, so this pose precess will try to fill those blank by calculate from near by boxes.

In [ ]:
import pandas as pd

model_name = "yolov8m"

df = pd.read_csv(model_name + "_before.csv")

List = []

for i in range(len(df)):
    Save = df["type"][i] + "," + df["name"][i]

    if Save not in List:
        List.append(Save)

C = 0
xmin = []
ymin = []
xmax = []
ymax = []
for i in range(len(List)):
    Type = List[i].split(",")[0]
    Name = List[i].split(",")[1]

    df_video = df.loc[(df["name"] == Name) & (df["type"] == Type)]

    df_video = df_video.reset_index()

    List_id = []
    for z in range(len(df_video)):
        Save = df_video["id"][z]
        if Save not in List_id:
            List_id.append(Save)

    for x in range(len(List_id)):
        df_check = df_video.loc[(df_video["id"] == x)]
        df_check = df_check.reset_index()

        List_id_2 = []
        for z in range(len(df_check)):
            Save = df_check["xmin"][z]
            if Save not in List_id_2:
                List_id_2.append(Save)

        if len(List_id_2) == 2:
            for j in range(len(df_check)):
                xmin.append(df_check["xmin"][0])
                ymin.append(df_check["ymin"][0])
                xmax.append(df_check["xmax"][0])
                ymax.append(df_check["ymax"][0])

        else:
            for j in range(len(df_check)):
                if df_check["xmin"][j] == -1:
                    Second = False
                    Length = 0
                    for k in range(len(df_check) - j - 1):
                        if df_check["xmin"][j + k + 1] == -1:
                            pass
                        else:
                            Second = df_check["xmin"][j + k + 1]
                            Length = k + 1
                            break
                    if Second == False:
                        F_xmin = df_check["xmin"][j - 2]
                        F_ymin = df_check["ymin"][j - 2]
                        F_xmax = df_check["xmax"][j - 2]
                        F_ymax = df_check["ymax"][j - 2]

                        S_xmin = df_check["xmin"][j - 1]
                        S_ymin = df_check["ymin"][j - 1]
                        S_xmax = df_check["xmax"][j - 1]
                        S_ymax = df_check["ymax"][j - 1]

                        xmin.append(int(S_xmin + (S_xmin - F_xmin)))
                        ymin.append(int(S_ymin + (S_ymin - F_ymin)))
                        xmax.append(int(S_xmax + (S_xmax - F_xmax)))
                        ymax.append(int(S_ymax + (S_ymax - F_ymax)))

                        df_check.loc[j, "xmin"] = int(S_xmin + (S_xmin - F_xmin))
                        df_check.loc[j, "ymin"] = int(S_ymin + (S_ymin - F_ymin))
                        df_check.loc[j, "xmax"] = int(S_xmax + (S_xmax - F_xmax))
                        df_check.loc[j, "ymax"] = int(S_ymax + (S_ymax - F_ymax))

                    else:
                        F_xmin = df_check["xmin"][j - 1]
                        F_ymin = df_check["ymin"][j - 1]
                        F_xmax = df_check["xmax"][j - 1]
                        F_ymax = df_check["ymax"][j - 1]

                        S_xmin = df_check["xmin"][j + Length]
                        S_ymin = df_check["ymin"][j + Length]
                        S_xmax = df_check["xmax"][j + Length]
                        S_ymax = df_check["ymax"][j + Length]

                        xmin.append(int(F_xmin + ((S_xmin - F_xmin) / (Length + 1))))
                        ymin.append(int(F_ymin + ((S_ymin - F_ymin) / (Length + 1))))
                        xmax.append(int(F_xmax + ((S_xmax - F_xmax) / (Length + 1))))
                        ymax.append(int(F_ymax + ((S_ymax - F_ymax) / (Length + 1))))

                        df_check.loc[j, "xmin"] = F_xmin + ((S_xmin - F_xmin) / (Length + 1))
                        df_check.loc[j, "ymin"] = F_ymin + ((S_ymin - F_ymin) / (Length + 1))
                        df_check.loc[j, "xmax"] = F_xmax + ((S_xmax - F_xmax) / (Length + 1))
                        df_check.loc[j, "ymax"] = F_ymax + ((S_ymax - F_ymax) / (Length + 1))

                else:
                    xmin.append(df_check["xmin"][j])
                    ymin.append(df_check["ymin"][j])
                    xmax.append(df_check["xmax"][j])
                    ymax.append(df_check["ymax"][j])

df["xmin"] = xmin
df["ymin"] = ymin
df["xmax"] = xmax
df["ymax"] = ymax

df.to_csv(model_name + "_after.csv", index=False)

Just make every answer to int

In [ ]:
import pandas as pd

model_name = "yolov8m"

df = pd.read_csv(model_name + "_after.csv")

xmin = []
ymin = []
xmax = []
ymax = []

for i in range(len(df)):
    xmin.append(int(df["xmin"][i]))
    ymin.append(int(df["ymin"][i]))
    xmax.append(int(df["xmax"][i]))
    ymax.append(int(df["ymax"][i]))

df["xmin"] = xmin
df["ymin"] = ymin
df["xmax"] = xmax
df["ymax"] = ymax

df.to_csv(model_name + "_after.csv", index=False)

Note: We create predicted_output by copy after.csv and paste into txt file and del column name.

# [5] Ensemble

Combine result of all 3 models

In [ ]:
import pandas as pd
import os

df_1 = pd.read_csv("yolov5_before.csv")
df_2 = pd.read_csv("yolov8_before.csv")
df_3 = pd.read_csv("yolov6_before.csv")

xmin = []
ymin = []
xmax = []
ymax = []

for i in range(len(df_1)):
    if df_1["xmin"][i] != -1:
        xmin.append(int(df_1["xmin"][i]))
        ymin.append(int(df_1["ymin"][i]))
        xmax.append(int(df_1["xmax"][i]))
        ymax.append(int(df_1["ymax"][i]))
    elif df_2["xmin"][i] != -1:
        xmin.append(int(df_2["xmin"][i]))
        ymin.append(int(df_2["ymin"][i]))
        xmax.append(int(df_2["xmax"][i]))
        ymax.append(int(df_2["ymax"][i]))
    else:
        xmin.append(int(df_3["xmin"][i]))
        ymin.append(int(df_3["ymin"][i]))
        xmax.append(int(df_3["xmax"][i]))
        ymax.append(int(df_3["ymax"][i]))

df_1["xmin"] = xmin
df_1["ymin"] = ymin
df_1["xmax"] = xmax
df_1["ymax"] = ymax

df_1.to_csv("ens_1_before.csv", index=False)